In [1]:

import sys
import tensorflow as tf
import pandas as pd
import numpy as np
import Config
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
#import doc2vec_impl
import gensim

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:

batch_size=10
embedding_filename = "glove_6B_200d.txt"
training_epochs=1;


In [3]:
sys.path

['',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/extensions',
 '/Users/yashd/.ipython']

In [4]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]



In [5]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();




d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models/doc2vec.model')
#start testing
#printing the vector of document at index 1 in docLabels
docvec = d2v_model.docvecs[files[0]]
print(docvec)


[-1.9957069  -1.7302711  -1.2416555  -1.25041    -0.8760755   1.4730991
  1.2059473   0.38232905 -1.6187527   1.305874    1.1770602   2.308416
  0.02343252 -0.2502587   1.7815354  -1.332049    0.38943136 -2.052053
 -1.3515872  -1.5760187  -0.59295475 -0.66141963  1.3531227   0.47351202
  0.80553395  2.9402657   2.2971811   0.9491949   4.030078    0.6117527
 -0.27292827 -1.1015353  -0.3637588  -1.0808604  -0.38145977 -2.5174286
  1.1072197   2.530051    2.5857248  -1.1959172   1.7793411   0.09467691
 -1.4371932  -2.9791405   1.0630684   0.9067003   0.43923157  3.4134736
  0.47711533 -1.3631905  -3.3734288  -0.2367886   0.3390342  -0.39412883
  0.5333661   0.7227717   0.55354226  0.35878623  0.932402    2.4869206
 -0.55152404  2.4435148  -0.3364468  -1.9685099  -1.7815394  -0.6749915
  2.4418046  -2.3903039   1.1918342   0.36871168 -1.7374214   0.7597355
  1.6034806  -0.9551855   0.8539238  -1.5502286   0.8968359  -0.62780535
 -1.7566789   4.206336    0.92475533  0.4445385   1.4446664   

In [6]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [7]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [8]:

embedding_array,word_index = load_embeddings(embedding_filename)
print("Loaded embeddings")

words_data=pd.read_csv("words_data.csv");
catchphrases_data=pd.read_csv("catch_data.csv")
print("reading data completed");

Total Words in DataSet: 400000
Loaded embeddings
reading data completed


In [9]:
print(word_index['unk'])


201534


In [10]:
def get_statements(file):
    with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    #catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    #sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    sentences+=tokenizer.tokenize(sentence.text)
    
    
    sentences = [w for w in sentences if not w in stop_words_list] 
    catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases


    

In [11]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

In [12]:
def lookup_indexes(sentences,catchphrases):
    
    sentence_indexes=[];
    catchphrase_indexes=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in catchphrases:
        if word in word_index:
            catchphrase_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            catchphrase_indexes.append(word_index['unk']);
            
    return sentence_indexes,catchphrase_indexes;

In [27]:

class CatchPhraseExtractor(object):

    def __init__(self, graph, embedding_array):
        self.total_loss=0.0
        self.build_graph(graph, embedding_array)

    def build_graph(self, graph, embedding_array):
        print("generating graph");

        with graph.as_default():

            #self.embeddings = tf.Variable(embedding_array, dtype=tf.float32);
            W = tf.Variable(tf.constant(0.0, shape=[400000, 200 ]),trainable=True, name="W")
            self.embedding_placeholder = tf.placeholder(tf.float32, [400000, 200])
            self.embedding_init = W.assign(self.embedding_placeholder)
            
            
            
            self.train_inputs = tf.placeholder(tf.int32, shape=[None,5])
            self.train_labels = tf.placeholder(tf.int32, shape=[None,5])
            
            sentence_vectors=tf.nn.embedding_lookup(W, self.train_inputs)
            catchphrase_vectors=tf.nn.embedding_lookup(W, self.train_labels)
            
            
            sentence_vectors = tf.reshape(sentence_vectors, [tf.shape(sentence_vectors)[0], 1000])
            catchphrase_vectors = tf.reshape(catchphrase_vectors, [tf.shape(catchphrase_vectors)[0],1000])
            
            conv_filter=tf.Variable(tf.truncated_normal((1000,200), stddev=0.1));
            
            word_features=tf.nn.relu(tf.matmul(sentence_vectors, conv_filter));
            catchphrase_features=tf.nn.relu(tf.matmul(catchphrase_vectors, conv_filter));
            
            doc_vector=tf.reduce_max(word_features,reduction_indices=[0],keepdims=True);
            print(doc_vector)
            
            sentence_doc_dummy=tf.tile(doc_vector, [tf.shape(word_features)[0], 1])
            catchphrase_doc_dummy=tf.tile(doc_vector, [tf.shape(catchphrase_features)[0], 1])

            
            sentence_tensor = tf.reshape(tf.concat([word_features, sentence_doc_dummy],1),[tf.shape(word_features)[0],400])
            catchphrase_tensor = tf.reshape(tf.stack([catchphrase_features, catchphrase_doc_dummy], 1),[tf.shape(catchphrase_features)[0],400])
            
            print(sentence_tensor)
            
            weights_input=tf.Variable(tf.truncated_normal((400,200), stddev=0.1));
            weights_input_2=tf.Variable(tf.truncated_normal((200,1), stddev=0.1));
            
            
            sentence_values=self.MLP(sentence_tensor,weights_input,weights_input_2);
            catchphrase_values=self.MLP(catchphrase_tensor,weights_input,weights_input_2);
            
            print(sentence_tensor)
            #print_op=tf.print(doc_vector);
            
            

            self.loss =tf.reduce_mean(sentence_values)-tf.reduce_mean(catchphrase_values)
            
            
            self.loss=tf.maximum(0.0,self.loss)
            optimizer = tf.train.AdamOptimizer(0.001)
            grads = optimizer.compute_gradients(tf.scalar_mul(100000,self.loss))
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)
            

            self.init = tf.global_variables_initializer()
            
    def MLP(self,x,weights_input,weights_input_2):
        h1=(tf.matmul(x, weights_input));
        h1=tf.add(h1,0.5);
        h1=tf.nn.tanh(h1);
        
        h2=tf.matmul(h1,weights_input_2);
        h2=tf.add(h2,0.5)
        h2=tf.nn.sigmoid(h2)
        
        
        return h2;
    
    def gradient_descent(loss_val):
        print("Optmising gradients");   
    
    def train(self, sess, num_steps):
        
        self.init.run()
        print("Initailized")
        total_loss=0.0;
        for file in files[:1000]:
            sentences,catchphrases=get_statements(file);
            sentence_indexes,catch_phrase_indexes=lookup_indexes(sentences,catchphrases)
            
            sentence_index_matrix=prepare_index_matrices(sentence_indexes);
            catchphrase_index_matrix=prepare_index_matrices(catch_phrase_indexes);
            
            
            sentence_index_matrix=np.asarray(sentence_index_matrix)
            catchphrase_index_matrix=np.asarray(catchphrase_index_matrix)
        
            #sentence_indexes=np.reshape(sentence_indexes,(sentence_indexes.shape[0],1))
            #catch_phrase_indexes=np.reshape(catch_phrase_indexes,(catch_phrase_indexes.shape[0],1))
            
            print(sentence_index_matrix.shape)
            feed_dict = {self.train_inputs: sentence_index_matrix, self.train_labels: catchphrase_index_matrix}
            #_, loss_val = sess.run([self.app, self.loss], feed_dict=feed_dict)  
            loss_val = sess.run([self.loss], feed_dict=feed_dict)
            #total_loss=total_loss+loss_val;
        
            print("Train Finished.",loss_val)
        print("total_loss_collected",total_loss);
        
        
        #self.gradient_descent(total_loss,tape.gradient(loss_value, self.trainable_variables))
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [28]:
def get_features(inputs):
    out=np.zeros((len(inputs),200));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,200));
        out[i]=c;
    
    return np.transpose(out);

In [29]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph, embedding_array)


generating graph
Tensor("Max:0", shape=(1, 200), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 400), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 400), dtype=float32)


In [30]:
with tf.Session(graph=graph) as sess:
    model.load_embeddings(embedding_array)    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    global_step = tf.train.get_or_create_global_step()
    model.train(sess, 10)   
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

Initailized
(2964, 5)
Train Finished. [0.0]
(2659, 5)
Train Finished. [0.0]
(5213, 5)
Train Finished. [0.0]
(5770, 5)
Train Finished. [0.0]
(2029, 5)
Train Finished. [5.9604645e-08]
(2934, 5)
Train Finished. [0.0]
(1419, 5)
Train Finished. [1.3709068e-06]
(1950, 5)
Train Finished. [2.3841858e-07]
(1000, 5)
Train Finished. [5.364418e-07]
(1117, 5)
Train Finished. [7.748604e-07]
(10295, 5)
Train Finished. [0.0]
(5239, 5)
Train Finished. [0.0]
(5283, 5)
Train Finished. [0.0]
(3078, 5)
Train Finished. [0.0]
(882, 5)
Train Finished. [1.1920929e-07]
(1778, 5)
Train Finished. [6.556511e-07]
(4376, 5)
Train Finished. [0.0]
(2236, 5)
Train Finished. [0.0]
(9017, 5)
Train Finished. [0.0]
(1736, 5)
Train Finished. [7.748604e-07]
(7737, 5)
Train Finished. [0.0]
(2710, 5)
Train Finished. [0.0]
(1260, 5)
Train Finished. [1.0728836e-06]
(3004, 5)
Train Finished. [0.0]
(2778, 5)
Train Finished. [0.0]
(2693, 5)
Train Finished. [0.0]
(1244, 5)
Train Finished. [1.0728836e-06]
(361, 5)
Train Finished. [0.

(5295, 5)
Train Finished. [0.0]
(1129, 5)
Train Finished. [8.34465e-07]
(1207, 5)
Train Finished. [9.536743e-07]
(741, 5)
Train Finished. [0.0]
(1724, 5)
Train Finished. [8.34465e-07]
(1704, 5)
Train Finished. [8.940697e-07]
(1950, 5)
Train Finished. [2.3841858e-07]
(1664, 5)
Train Finished. [1.013279e-06]
(2684, 5)
Train Finished. [0.0]
(3078, 5)
Train Finished. [0.0]
(3904, 5)
Train Finished. [0.0]
(1023, 5)
Train Finished. [5.9604645e-07]
(645, 5)
Train Finished. [0.0]
(943, 5)
Train Finished. [2.9802322e-07]
(539, 5)
Train Finished. [0.0]
(11769, 5)
Train Finished. [0.0]
(2119, 5)
Train Finished. [0.0]
(20463, 5)
Train Finished. [2.1100044e-05]
(6768, 5)
Train Finished. [0.0]
(212, 5)
Train Finished. [0.0]
(1205, 5)
Train Finished. [9.536743e-07]
(2053, 5)
Train Finished. [0.0]
(472, 5)
Train Finished. [0.0]
(3681, 5)
Train Finished. [0.0]
(3030, 5)
Train Finished. [0.0]
(3002, 5)
Train Finished. [0.0]
(1489, 5)
Train Finished. [1.4305115e-06]
(14712, 5)
Train Finished. [7.0929527e

Train Finished. [0.0]
(7444, 5)
Train Finished. [0.0]
(5466, 5)
Train Finished. [0.0]
(2285, 5)
Train Finished. [0.0]
(1676, 5)
Train Finished. [9.536743e-07]
(1513, 5)
Train Finished. [1.4901161e-06]
(7210, 5)
Train Finished. [0.0]
(625, 5)
Train Finished. [0.0]
(1201, 5)
Train Finished. [1.013279e-06]
(630, 5)
Train Finished. [0.0]
(250, 5)
Train Finished. [0.0]
(1519, 5)
Train Finished. [1.4305115e-06]
(1033, 5)
Train Finished. [5.9604645e-07]
(2520, 5)
Train Finished. [0.0]
(7095, 5)
Train Finished. [0.0]
(2836, 5)
Train Finished. [0.0]
(6063, 5)
Train Finished. [0.0]
(893, 5)
Train Finished. [1.7881393e-07]
(3030, 5)
Train Finished. [0.0]
(5680, 5)
Train Finished. [0.0]
(4301, 5)
Train Finished. [0.0]
(1465, 5)
Train Finished. [1.4305115e-06]
(1868, 5)
Train Finished. [4.172325e-07]
(516, 5)
Train Finished. [0.0]
(395, 5)
Train Finished. [0.0]
(2488, 5)
Train Finished. [0.0]
(2890, 5)
Train Finished. [0.0]
(444, 5)
Train Finished. [0.0]
(989, 5)
Train Finished. [4.7683716e-07]
(39

Train Finished. [0.0]
(348, 5)
Train Finished. [0.0]
(4138, 5)
Train Finished. [0.0]
(1779, 5)
Train Finished. [6.556511e-07]
(411, 5)
Train Finished. [0.0]
(561, 5)
Train Finished. [0.0]
(1895, 5)
Train Finished. [3.5762787e-07]
(5531, 5)
Train Finished. [0.0]
(1021, 5)
Train Finished. [5.364418e-07]
(438, 5)
Train Finished. [0.0]
(452, 5)
Train Finished. [0.0]
(1029, 5)
Train Finished. [5.9604645e-07]
(1844, 5)
Train Finished. [4.7683716e-07]
(1430, 5)
Train Finished. [1.3709068e-06]
(634, 5)
Train Finished. [0.0]
(3878, 5)
Train Finished. [0.0]
(465, 5)
Train Finished. [0.0]
(3412, 5)
Train Finished. [0.0]
(2448, 5)
Train Finished. [0.0]
(1877, 5)
Train Finished. [4.172325e-07]
(288, 5)
Train Finished. [0.0]
(874, 5)
Train Finished. [1.1920929e-07]
(2272, 5)
Train Finished. [0.0]
(6408, 5)
Train Finished. [0.0]
(788, 5)
Train Finished. [0.0]
(4088, 5)
Train Finished. [0.0]
(2515, 5)
Train Finished. [0.0]
(2021, 5)
Train Finished. [5.9604645e-08]
(2338, 5)
Train Finished. [0.0]
(6683

Train Finished. [0.0]
(261, 5)
Train Finished. [0.0]
(2603, 5)
Train Finished. [0.0]
(7751, 5)
Train Finished. [0.0]
(1429, 5)
Train Finished. [1.3709068e-06]
(2164, 5)
Train Finished. [0.0]
(768, 5)
Train Finished. [0.0]
(954, 5)
Train Finished. [3.5762787e-07]
(3424, 5)
Train Finished. [0.0]
(404, 5)
Train Finished. [0.0]
(1446, 5)
Train Finished. [1.3709068e-06]
(305, 5)
Train Finished. [0.0]
(3243, 5)
Train Finished. [0.0]
(4306, 5)
Train Finished. [0.0]
(15650, 5)
Train Finished. [1.0073185e-05]
(844, 5)
Train Finished. [0.0]
(3677, 5)
Train Finished. [0.0]
(1126, 5)
Train Finished. [7.748604e-07]
(985, 5)
Train Finished. [4.7683716e-07]
(1918, 5)
Train Finished. [2.9802322e-07]
(731, 5)
Train Finished. [0.0]
(1444, 5)
Train Finished. [1.3709068e-06]
(714, 5)
Train Finished. [0.0]
(7241, 5)
Train Finished. [0.0]
(8526, 5)
Train Finished. [0.0]
(10596, 5)
Train Finished. [0.0]
(1237, 5)
Train Finished. [1.0728836e-06]
(5437, 5)
Train Finished. [0.0]
(1355, 5)
Train Finished. [1.251